In [1]:
import json
import pandas as pd

In [2]:
df = pd.read_csv('Data/assumptions.csv')

In [3]:
df.head()

,level1,level2,level3,level4,value
0,A,HR,Investment,a,1.0
1,A,HR,cash,a,2.0
2,A,LR,Investment,b,3.0
3,A,LR,cash,b,4.0
4,B,HR,Investment,a,5.0


In [4]:
indexed_df = df.set_index(['level1', 'level2', 'level3', 'level4'])

In [5]:
value_series = indexed_df['value']

In [6]:
unstacked_df = value_series.unstack(level=3)
unstacked_df

level4                      a    b
level1 level2 level3              
A      HR     Investment  1.0  NaN
              cash        2.0  NaN
       LR     Investment  NaN  3.0
              cash        NaN  4.0
B      HR     Investment  5.0  NaN
              cash        6.0  NaN
       LR     Investment  NaN  7.0
              cash        NaN  8.0

In [7]:
result = {}

In [8]:
# Create the top-level container
result["Assumptions"] = {}

In [9]:
# Create the top-level container
# Get unique level1 values
for level1_val in unstacked_df.index.get_level_values(0).unique():
    # Create level1 entry
    result["Assumptions"][level1_val] = {}

    # Get level2 values for this level1
    df_level1 = unstacked_df.xs(level1_val, level=0)

    for level2_val in df_level1.index.get_level_values(0).unique():
        # Create level2 entry
        result["Assumptions"][level1_val][level2_val] = {}

        # Get level3 values for this level2
        df_level2 = df_level1.xs(level2_val, level=0)

        # For each level3, add the level4 values
        for level3_val, row in df_level2.iterrows():
            result["Assumptions"][level1_val][level2_val][level3_val] = row.dropna().to_dict()

In [10]:
with open('Data/assumptions4.json', 'w') as f:
    json.dump(result, f, indent=2)

In [11]:
def df_to_nested_json(df, top_level_name, level_columns, value_column='value'):
    """
    Convert a DataFrame to a nested JSON structure.

    Parameters:
    ----------
    df : pandas.DataFrame
        Input DataFrame with hierarchical data
    top_level_name : str
        Name for the top-level container in the JSON
    level_columns : list
        List of column names to use as hierarchical levels
    value_column : str, default 'value'
        Name of the column containing the values

    Returns:
    -------
    dict
        Nested dictionary structure ready for JSON conversion
    """
    # Set up multi-index DataFrame
    indexed_df = df.set_index(level_columns)
    value_series = indexed_df[value_column]

    # Unstack the deepest level to create columns
    unstacked_df = value_series.unstack(level=len(level_columns)-1)

    # Create the result dictionary
    result = {top_level_name: {}}

    # Helper function to recursively build the nested structure
    def build_nested_dict(df, current_dict, levels_seen=0, max_levels=len(level_columns)-2):
        if levels_seen == max_levels:
            # At the deepest level, process rows
            for idx, row in df.iterrows():
                current_dict[idx] = row.dropna().to_dict()
        else:
            # At intermediate levels, create new dictionaries and recurse
            for val in df.index.get_level_values(0).unique():
                current_dict[val] = {}
                next_df = df.xs(val, level=0)
                build_nested_dict(next_df, current_dict[val], levels_seen+1, max_levels)

    # Start building from the top level
    build_nested_dict(unstacked_df, result[top_level_name])

    return result

In [12]:
# Create the JSON structure
result = df_to_nested_json(df, "Assumptions", ["level1", "level2", "level3", "level4"])

# Write to JSON file
with open('Data/assumptions5.json', 'w') as f:
    json.dump(result, f, indent=2)